# BBC Weather Data 
In this project we obtain the weather prediction data of any city using BBC weather website. 
First, we make an api call to obtain the location id of the city we want the weather data. Next using the location id we make connection with the bbc database to extract high, low and summary of the day's temperature for the next 14 days.

In [1]:
from urllib.parse import urlencode    
import requests       # To make connection with the webpage
from bs4 import BeautifulSoup as bs      # To parse the webpage
import pandas as pd                      
import datetime    

Making connection to obtain city id

In [2]:
city = 'Mumbai'     # City name 
location_url = 'https://locator-service.api.bbci.co.uk/locations?' +urlencode({   
      'api_key' : 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv' ,
      'stack' : 'aws' ,
      'locale' : 'en', 
      'filter' : 'international', 
      'place-types' : 'settlement,airport,district', 
      'order' : 'importance', 
      's' : city, 
      'a' : 'true', 
      'format' : 'json'
})

In [3]:
result = requests.get(location_url).json()    # Making an API call to get the location id of the city
result

{'response': {'results': {'results': [{'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '1275339',
     'language': 'en',
     'latitude': 19.07283,
     'longitude': 72.88261,
     'name': 'Mumbai',
     'placeType': 'settlement',
     'timezone': 'Asia/Kolkata'}],
   'totalResults': 1}}}

In [4]:
city_id = result['response']['results']['results'][0]['id']     # Extracting the location id
city_id 

'1275339'

Getting Data based on city id

In [5]:
url = 'https://www.bbc.com/weather/' + city_id        
data = requests.get(url)      # Making connection with the database with location id
soup = bs(data.content, 'html.parser')    # Using Beautiful soup to parse the webpage
high_temp = soup.find_all('span', 'wr-day-temperature__high-value')     # Getting high temperature in celsius and fehrehite for 14 days
low_temp = soup.find_all('span', 'wr-day-temperature__low-value')       # Getting low temperature in celsius and fehrehite for 14 days
weather_summary = soup.find_all('div', 'wr-day__details__weather-type-description')   # Getting summary for 14 days


In [6]:
high_c = [int(x.text.split()[0][:-1]) for x in high_temp]    # Extracting only the celsius temperature
low_c = [int(x.text.split()[0][:-1]) for x in low_temp]
summary = [x.text for x in weather_summary]     

In [7]:
if len(high_c) != len(low_c):
    high_c = [low_c[0]] + high_c  # this is only when the high value of the day is not-available

In [8]:
date = [(datetime.date.today() + datetime.timedelta(days=i)).strftime('%d-%m-%Y') for i in range(14)]   # List of date for next 14 days
date

['07-01-2022',
 '08-01-2022',
 '09-01-2022',
 '10-01-2022',
 '11-01-2022',
 '12-01-2022',
 '13-01-2022',
 '14-01-2022',
 '15-01-2022',
 '16-01-2022',
 '17-01-2022',
 '18-01-2022',
 '19-01-2022',
 '20-01-2022']

Creating Pandas DataFrame

In [9]:
weather_df = pd.DataFrame({'Date' : date, 'High' : high_c, 'Low' : low_c, 'Summary' : summary})   # Creating pandas dataframe from the lists
weather_df.head(15)
weather_df.style.set_caption(city + ' Weather Prediction for next 14 days')

,Date,High,Low,Summary
0,07-01-2022,29,21,Sunny intervals and a gentle breeze
1,08-01-2022,27,20,Light cloud and a gentle breeze
2,09-01-2022,27,18,Sunny intervals and a gentle breeze
3,10-01-2022,26,18,Sunny intervals and a gentle breeze
4,11-01-2022,28,19,Sunny intervals and a gentle breeze
5,12-01-2022,28,20,Sunny intervals and a gentle breeze
6,13-01-2022,28,20,Sunny intervals and a gentle breeze
7,14-01-2022,28,19,Sunny intervals and a gentle breeze
8,15-01-2022,29,20,Sunny intervals and a gentle breeze
9,16-01-2022,30,20,Sunny intervals and a gentle breeze
